In [2]:
# STEP 1: UPLOAD YOUR AUDIO FILE AND GET URL
import requests
import sys
sys.path.append("/configure")
from configure import auth_key

filename = '/Users/rickypinto/Documents/UpGrad/audio-to-text/SteveJobsSpeech_64kb.mp3' 
api_key =  auth_key
upload_endpoint = 'https://api.assemblyai.com/v2/upload'

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as f:
        while True:
            data = f.read(chunk_size)
            if not data:
                break
            yield data


headers = {'authorization': api_key,
           'content-type': 'application/json'}
response = requests.post(upload_endpoint,
                         headers=headers,
                         data=read_file(filename))

audio_url = response.json()['upload_url']

In [4]:
# STEP 2: GET THE TRANSCRIPT REQUEST
transcript_endpoint = "https://api.assemblyai.com/v2/transcript"

response = requests.post(transcript_endpoint,
                         headers=headers,
                         json={
                             "audio_url": audio_url,
                             "auto_chapters": True
                         })
transcript_id = response.json()['id']

In [5]:
# STEP 3: SAVE THE TRANSCRIPT AND SUMMARY
import os
import sys
import time
import json

polling_endpoint =  os.path.join(transcript_endpoint, transcript_id)

status = ''
while status != 'completed':
    response_result = requests.get(
        polling_endpoint,
        headers=headers
    )
    status = response_result.json()['status']
    print(f'Status: {status}')

    if status == 'error':
        sys.exit('Audio file failed to process.')
    elif status != 'completed':
        time.sleep(10)


if status == 'completed':
    filename = transcript_id + '.txt'
    with open(filename, 'w') as f:
        f.write(response_result.json()['text'])

    filename = transcript_id + '_chapters.json'
    with open(filename, 'w') as f:
        chapters = response_result.json()['chapters']
        json.dump(chapters, f, indent=4)

    print('Transcript Saved')

Status: processing
Status: processing
Status: processing
Status: processing
Status: processing
Status: processing
Status: processing
Status: processing
Status: processing
Status: processing
Status: processing
Status: completed
Transcript Saved
